<a href="https://colab.research.google.com/github/devesh962/Covid_Data_Extraction/blob/master/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 2.5MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=2bc267a16e9e819df4c054482b2ac8c6aec041d697010d88932fd72ede75e0e2
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
import PyPDF2 #to handle pdf files
import os
from os import listdir
#nltk libraries to process the text
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import csv
from csv import writer

In [ ]:
#assuming that there is a folder coivd-19 which contains all the reports in pdf format

#to extract all the pdf files in a list
def getAll():

  #path of the folder where all pdf files are stored
  path="/content/covid-19"
  arr = os.listdir(path)
  
  return arr

In [ ]:
#preprocessing on the text obtained from pdf

#removing the stop words, punctuations and converting all the text to lowercase
def process_text(message, lower_case = True, stem = True, stop_words = True, gram = 2):
    if lower_case:
        message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w) >= 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    if stop_words:
        sw = stopwords.words('english')
        words = [word for word in words if word not in sw]
    if stem:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]   
    return words

In [ ]:
#to extract the data from pdf file
def extractData(arr):
  path="/content/covid-19"
  for i in arr:
    path2=os.path.join(path,i)
    pdfFileObj = open(path2, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #getting page indexed 0 if there are more than one page for loop can be used to iterate over all the pages
    pageObj = pdfReader.getPage(0)
    #extracting the text
    Pdf_String=pageObj.extractText()
    Pdf_String=Pdf_String.strip()
    Pdf_String=Pdf_String.replace("\n","")
    #calling function processed_text to process the text obtained from pdf
    processed_text=process_text(Pdf_String,gram=1,stem=False)
    final=processed_text
    row=[]
    #searching particular keywords in the text to extract the desired data
    #iterating a loop over all the text obtained from the pdf
    i=0
    while i<len(final):
      if final[i]=="status":
        if final[i+1]=="update":
      
          if i+4<len(final):
            row.append(final[i+2:i+5])
            i=i+4
      elif final[i]=="total":
        row.append(final[i+1])
        i+=1
      elif final[i]=="individuals":
        if final[i+1]=="confirmed":
          if final[i+2]=="positive":
            row.append(final[i-1])
      elif final[i]=="samples":
        if final[i+1]=="reported":
          row.append(final[i-1])
      elif final[i]=="reported":
        if final[i+2]=="positive":
          row.append(final[i+1])
          i+=2
    
      i+=1
    data.append(row)
  return data

In [ ]:
#to convert the format of date obtained from the pdf file
from datetime import datetime

In [ ]:
# to process the data extracted from pdf file
def process_data(data):
  for i in range(len(data)):
    data[i][0]=' '.join(data[i][0])
    #converting the format of date
    data[i][0]=datetime.strptime(data[i][0], '%d %B %Y').strftime('%Y/%m/%d')
    for j in range(1,len(data[i])):
      #removing all the stopwords if there in the data extracted
      data[i][j]=data[i][j].replace(",","")
      data[i][j]=int(data[i][j])
  return data

In [ ]:
#for appending the obtained data into csv file
def append_list_as_row(file_name, list_of_elem,j):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        if j==0:
          #if first column is appended then first append the tag names
          csv_writer = writer(write_obj)
          csv_writer.writerow(["Date","total_samples_test","total_positives","sample_reported","new_positives"])

        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
#to append data in csv file
def append_csv(data):
  path='/content/Covid-19.csv'
  for j,i in enumerate(data):
    append_list_as_row(path,i,j)


In [ ]:
#function calls

arr=getAll() #extract path of all the pdf files
data=[]
data=extractData(arr) #extract the data from pdf files
data=process_data(data) #process the data
append_csv(data)         #append into csv file
  

#assuming we have a csv file Covid-19.csv uploaded on colab or else we can make a csv file and append data in it